# Data Preparation and Cleaning

In this notebook, I clean the datasets and combine them into a single csv file that can be used later for feature generation.

In [26]:
# Import packages
import sys
sys.path.append('./')

import pandas as pd
import collegebasketball as cbb
cbb.__version__

'2023'

## Load in the Game Scores Data

First, we will load in the games scores data from csv files we created earlier. Later, we'll join this data to the team stats datasets. 

In [33]:
# Location of the data
scores_path = './Data/Scores/'

# Initialize some variables
scores_data = {}
year = 2024

# Load the scores datasets
scores_data = pd.read_csv(scores_path + str(year) + '_season.csv')

## Cleaning the Data

Next, we need to edit the school names in the kenpom, basic stats and T-Rank datasets to ensure that they match up with the school names from the scores dataset. We will verify that the names match using the `cbb.check_for_missing_names` command. It checks that each school name in the given team statistics dataset (kenpom, basic stats or T-Rank) is present in the game scores dataset.

In [34]:
# The location where the files will be saved
path = './Data/'
    
# Load this year's data and clean up the school names to match up with scores data
kenpom_data = pd.read_csv('{0}Kenpom/{1}_kenpom.csv'.format(path, year))
kenpom_data = cbb.update_kenpom(kenpom_data)
missing_kenpom = cbb.check_for_missing_names(scores_data, kenpom_data, False)
kenpom_data = kenpom_data[~kenpom_data['Team'].isin(missing_kenpom)]
assert len(cbb.check_for_missing_names(scores_data, kenpom_data, False)) == 0

# TRank data
TRank_data =  pd.read_csv('{0}TRank/{1}_TRank.csv'.format(path, year))
TRank_data = cbb.update_TRank(TRank_data)
missing_trank = cbb.check_for_missing_names(scores_data, TRank_data, False)
TRank_data = TRank_data[~TRank_data['Team'].isin(missing_trank)]
assert len(cbb.check_for_missing_names(scores_data, TRank_data, False)) == 0

# Basic stats data
stats_data =  pd.read_csv('{0}SportsReference/{1}_stats.csv'.format(path, year))
stats_data = stats_data.rename(index=str, columns={'School': 'Team'})
stats_data = cbb.update_basic(stats_data)
missing_stats = cbb.check_for_missing_names(scores_data, stats_data, False)
stats_data = stats_data[~stats_data['Team'].isin(missing_stats)]
assert len(cbb.check_for_missing_names(scores_data, stats_data, False)) == 0

In [35]:
# Lets take a quick look at one of the datasets
kenpom_data.head()

,Rank,Team,Seed,Conf,Wins,Losses,AdjEM,AdjO,AdjO Rank,AdjD,...,Luck,Luck Rank,OppAdjEM,OppAdjEM Rank,OppO,OppO Rank,OppD,OppD Rank,NCSOS AdjEM,NCSOS AdjEM Rank
0,1,UConn,1.0,BE,31,3,31.67,126.4,1,94.7,...,0.047,72,10.04,42,111.4,47,101.4,37,-3.23,284
1,2,Houston,1.0,B12,30,4,31.45,118.7,18,87.3,...,0.053,59,11.60,15,111.8,38,100.2,7,-0.78,227
2,3,Purdue,1.0,B10,29,4,29.07,125.0,4,95.9,...,0.045,76,13.81,4,114.1,5,100.3,8,10.36,13
3,4,Auburn,4.0,SEC,27,7,28.87,120.6,10,91.7,...,-0.067,324,9.69,49,111.9,34,102.2,71,1.65,148
4,5,Arizona,2.0,P12,25,8,26.77,121.2,8,94.4,...,-0.043,287,11.05,24,112.0,31,101.0,23,10.60,11


## Joining the Datasets

Now that the school names from each data set matches up, we can join the kenpom and score data to form a single csv file. 

In [36]:
# Save the paths to the data 
save_path = './Data/Combined_Data/Kenpom.csv'
    
# Join the dataframes to get kenpom for both home and away team
kenpom_df = pd.merge(scores_data, kenpom_data, left_on='Home', right_on='Team', sort=False)
kenpom_df = pd.merge(kenpom_df, kenpom_data, left_on='Away', right_on='Team', 
                     suffixes=('_Home', '_Away'), sort=False)

# Add a column to indicate the year
kenpom_df.insert(0, 'Year', year)
        
# Combine the data for every year and save to csv
# all_kenpom = pd.read_csv(save_path)
# kenpom_df = pd.concat([all_kenpom, kenpom_df])
kenpom_df.to_csv(save_path, index=False)
    
# Lets take a look at the data set
print("There are {} games in the Kenpom dataset.".format(len(kenpom_df)))
print("There are {} NCAA Tournament games in the Kenpom dataset.".format(len(cbb.filter_tournament(kenpom_df))))
kenpom_df.head()

There are 5528 games in the Kenpom dataset.
There are 5498 NCAA Tournament games in the Kenpom dataset.


,Year,Home,Away,Home_Score,Away_Score,Tournament,Rank_Home,Team_Home,Seed_Home,Conf_Home,...,Luck_Away,Luck Rank_Away,OppAdjEM_Away,OppAdjEM Rank_Away,OppO_Away,OppO Rank_Away,OppD_Away,OppD Rank_Away,NCSOS AdjEM_Away,NCSOS AdjEM Rank_Away
0,2024,North Carolina Central,Kansas,56.0,99.0,NaN,255,North Carolina Central,NaN,MEAC,...,0.048,69,13.57,6,112.8,12,99.2,2,5.25,68
1,2024,Dartmouth,Duke,54.0,92.0,NaN,336,Dartmouth,NaN,Ivy,...,-0.077,334,8.65,75,110.6,70,102.0,64,0.22,200
2,2024,Samford,Purdue,45.0,98.0,NaN,81,Samford,13.0,SC,...,0.045,76,13.81,4,114.1,5,100.3,8,10.36,13
3,2024,James Madison,Michigan State,79.0,76.0,NaN,57,James Madison,12.0,SB,...,-0.104,355,12.40,9,114.2,4,101.8,57,4.53,79
4,2024,Northern Illinois,Marquette,70.0,92.0,NaN,306,Northern Illinois,NaN,MAC,...,0.036,96,12.69,8,113.1,8,100.4,10,8.34,23


Now we will clean up the team names in the T-Rank data and join it with the game scores data. Additionally, we need to join these data sets with the team Kenpom statistics. This join is necessary because we need to use the Tournament seed attribute in order to clean up the march dataset to only include NCAA Tournament games. It will also be beneficial down the road, during feature generation, for us to have the Kenpom AdjEM and W/L stats for each team as a way to judge what outcome of a game is considered an upset.

In [37]:
save_path = './Data/Combined_Data/TRank.csv'

# Get only the columns we need from the kenpom data
kp = kenpom_data[['Team', 'AdjEM', 'Seed']]

# Join the dataframes to get TRank data and kenpom (seed, adj_em) for both home and away team
TRank_df = pd.merge(scores_data, TRank_data, left_on='Home', right_on='Team', sort=False)
TRank_df = pd.merge(TRank_df, TRank_data, left_on='Away', right_on='Team', 
                         suffixes=('_Home', '_Away'), sort=False)
TRank_df = pd.merge(TRank_df, kp, left_on='Home', right_on='Team', sort=False)
TRank_df = pd.merge(TRank_df, kp, left_on='Away', right_on='Team', 
                    suffixes=('_x', '_y'), sort=False)

# Add a column to indicate the year
TRank_df.insert(0, 'Year', year)
    
# Combine the data for every year and save to csv
#all_TRank = pd.read_csv(save_path)
TRank_df.rename(columns={'Team_Home.1': 'Team_Home', 'Team_Away.1': 'Team_Away'}, inplace=True)
TRank_df.rename(columns={'AdjEM_x': 'AdjEM_Home', 'AdjEM_y': 'AdjEM_Away', 'Seed_x': 'Seed_Home', 'Seed_y': 'Seed_Away'}, inplace=True)
#TRank_df = pd.concat([all_TRank, TRank_df])
TRank_df.to_csv(save_path, index=False)
    
# Lets take a look at one of the data sets
print("There are {} games in the T-Rank dataset.".format(len(TRank_df)))
print("There are {} NCAA Tournament games in the T-Rank dataset.".format(len(cbb.filter_tournament(TRank_df))))
TRank_df.head()

There are 5528 games in the T-Rank dataset.
There are 5498 NCAA Tournament games in the T-Rank dataset.


,Year,Home,Away,Home_Score,Away_Score,Tournament,Rk_Home,Team_Home,Conf_Home,G_Home,...,Adj T._Away,Adj T. Rank_Away,WAB_Away,WAB Rank_Away,Team_x,AdjEM_Home,Seed_Home,Team_y,AdjEM_Away,Seed_Away
0,2024,North Carolina Central,Kansas,56.0,99.0,NaN,265,North Carolina Central,MEAC,27,...,68.9,93,4.10,14,North Carolina Central,-6.74,NaN,Kansas,18.96,4.0
1,2024,Dartmouth,Duke,54.0,92.0,NaN,332,Dartmouth,Ivy,25,...,66.9,216,3.20,17,Dartmouth,-16.98,NaN,Duke,24.84,4.0
2,2024,Samford,Purdue,45.0,98.0,NaN,89,Samford,SC,32,...,67.6,167,10.90,1,Samford,9.87,13.0,Purdue,29.07,1.0
3,2024,James Madison,Michigan State,79.0,76.0,NaN,71,James Madison,SB,33,...,65.3,303,0.15,53,James Madison,12.87,12.0,Michigan State,20.01,9.0
4,2024,Northern Illinois,Marquette,70.0,92.0,NaN,302,Northern Illinois,MAC,29,...,69.1,86,6.10,6,Northern Illinois,-11.56,NaN,Marquette,22.19,2.0


Lastly, we will run the same process for the basic statistics as we did for the T-Rank data.

In [38]:
save_path = './Data/Combined_Data/Basic.csv'
    
# Get only the columns we need from the kenpom data
kp = kenpom_data[['Team', 'AdjEM', 'Seed', 'Wins', 'Losses']]

# Join the dataframes to get basic statistics data and kenpom (seed, adj_em) for both home and away team
basic_df = pd.merge(scores_data, stats_data, left_on='Home', right_on='Team', sort=False)
basic_df = pd.merge(basic_df, stats_data, left_on='Away', right_on='Team', 
                    suffixes=('_Home', '_Away'), sort=False)
basic_df = pd.merge(basic_df, kp, left_on='Home', right_on='Team', sort=False)
basic_df = pd.merge(basic_df, kp, left_on='Away', right_on='Team', 
                    suffixes=('_x', '_y'), sort=False)

# Add a column to indicate the year
basic_df.insert(0, 'Year', year)
    
# Combine the data for every year and save to csv
# all_basic = pd.read_csv(save_path)
basic_df.rename(columns={'Team_Home.1': 'Team_Home', 'Team_Away.1': 'Team_Away'}, inplace=True)
basic_df.rename(columns={'AdjEM_x': 'AdjEM_Home', 'AdjEM_y': 'AdjEM_Away'}, inplace=True)
basic_df.rename(columns={'Seed_x': 'Seed_Home', 'Seed_y': 'Seed_Away'}, inplace=True)
basic_df.rename(columns={'Wins_x': 'Wins_Home', 'Wins_y': 'Wins_Away'}, inplace=True)
basic_df.rename(columns={'Losses_x': 'Losses_Home', 'Losses_y': 'Losses_Away'}, inplace=True)
# basic_df = pd.concat([all_basic, basic_df])
basic_df.to_csv(save_path, index=False)
    
# Lets take a look at one of the data sets
print("There are {} games in the regular season basic statistics dataset.".format(len(basic_df)))
print("There are {} NCAA tournament games in the basic statistics dataset.".format(len(cbb.filter_tournament(basic_df))))
basic_df.head()

There are 5501 games in the regular season basic statistics dataset.
There are 5471 NCAA tournament games in the basic statistics dataset.


,Year,Home,Away,Home_Score,Away_Score,Tournament,Team_Home,G_Home,SRS_Home,SOS_Home,...,Team_x,AdjEM_Home,Seed_Home,Wins_Home,Losses_Home,Team_y,AdjEM_Away,Seed_Away,Wins_Away,Losses_Away
0,2024,North Carolina Central,Kansas,56.0,99.0,NaN,North Carolina Central,31,-7.79,-8.75,...,North Carolina Central,-6.74,NaN,18,13,Kansas,18.96,4.0,22,10
1,2024,Dartmouth,Duke,54.0,92.0,NaN,Dartmouth,27,-13.82,-1.14,...,Dartmouth,-16.98,NaN,6,21,Duke,24.84,4.0,24,8
2,2024,Samford,Purdue,45.0,98.0,NaN,Samford,34,6.23,-3.74,...,Samford,9.87,13.0,29,5,Purdue,29.07,1.0,29,4
3,2024,James Madison,Michigan State,79.0,76.0,NaN,James Madison,34,8.65,-4.84,...,James Madison,12.87,12.0,31,3,Michigan State,20.01,9.0,19,14
4,2024,Northern Illinois,Marquette,70.0,92.0,NaN,Northern Illinois,31,-8.99,-1.19,...,Northern Illinois,-11.56,NaN,11,20,Marquette,22.19,2.0,25,9
